In [1]:
# NOTE: this clashes with smartnoise! dunno how to deal with, but we use this opacus
# for fttransformer, so we need to install it here
# ! pip install opacus==1.5.2

In [2]:
import numpy as np
import pandas as pd
import torch

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import ydnpd

from ydnpd.datasets.loader import load_dataset, split_train_eval_datasets

from ydnpd.pretraining.ft_transformer import FTTransformerModel
from ydnpd.pretraining.utils import preprocess_acs_for_ft_transformer, preprocess_acs_for_classification, print_model_performance

from pathlib import Path

LLM_PATH = "llm_datasets"
DATA_PATH = 'ydnpd/datasets/data'


In [3]:
LLM_EXPERIMENTS = {
    experiment_name:
    [
        (f"{experiment_name}/{path.stem}", LLM_PATH)
        for path in Path(f"{LLM_PATH}/{experiment_name}").glob("*.csv")
        ]
        for experiment_name in ["acs"]
}


EXPERIMENTS = {
    experiment_name:
    [
        (f"{experiment_name}/{path.stem}", DATA_PATH)
        for path in Path(f"{DATA_PATH}/{experiment_name}").glob("*.csv")
        ]
        for experiment_name in ["acs"]
}

In [4]:
real_dataset, real_processed_schema, real_domain = load_dataset('acs/national', path=DATA_PATH)
llm_dataset, llm_processed_schema, llm_domain = load_dataset(LLM_EXPERIMENTS['acs'][0][0], path=LLM_PATH)

### Baseline, non-private

In [5]:
real_df = preprocess_acs_for_classification(real_dataset)
X = real_df.drop('y', axis=1)
y = real_df['y']

X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(X, y, test_size=0.2)

# naive classifier
print(f"Base accuracy: {y_train_real.sum() / len(y_train_real)}")
print(classification_report(y_train_real, np.zeros_like(y_train_real)))
print('AUC:', roc_auc_score(y_train_real, np.zeros_like(y_train_real)))

# random forest
print('Random Forest')
rf = RandomForestClassifier()
rf.fit(X_train_real, y_train_real)
y_pred_real = rf.predict(X_test_real)
print(classification_report(y_test_real, y_pred_real))
print('AUC:', roc_auc_score(y_test_real, y_pred_real))
print()

# xgboost
print('XGBoost')
xgb = XGBClassifier()
xgb.fit(X_train_real, y_train_real)
y_pred_real = xgb.predict(X_test_real)
print(classification_report(y_test_real, y_pred_real))
print('AUC:', roc_auc_score(y_test_real, y_pred_real))
print()


Base accuracy: 0.5007688759034292
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      6493
           1       0.00      0.00      0.00      6513

    accuracy                           0.50     13006
   macro avg       0.25      0.50      0.33     13006
weighted avg       0.25      0.50      0.33     13006

AUC: 0.5
Random Forest


/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.66      0.67      0.66      1636
           1       0.66      0.65      0.66      1616

    accuracy                           0.66      3252
   macro avg       0.66      0.66      0.66      3252
weighted avg       0.66      0.66      0.66      3252

AUC: 0.6592434457382169

XGBoost
              precision    recall  f1-score   support

           0       0.68      0.68      0.68      1636
           1       0.68      0.68      0.68      1616

    accuracy                           0.68      3252
   macro avg       0.68      0.68      0.68      3252
weighted avg       0.68      0.68      0.68      3252

AUC: 0.6786603706214142



### Privacy, with FTTransformer

In [6]:
(
    X_cat_train,
    X_cont_train,
    X_cat_valid,
    X_cont_valid,
    y_train,
    y_valid,
    cat_cardinalities,
    config
) = preprocess_acs_for_ft_transformer(real_dataset)

(
    X_cat_train_llm,
    X_cont_train_llm,
    X_cat_valid_llm,
    X_cont_valid_llm,
    y_train_llm,
    y_valid_llm,
    cat_cardinalities_llm,
    _
) = preprocess_acs_for_ft_transformer(llm_dataset, config=config)


column SEX is categorical
column MSP is categorical
column RAC1P is categorical
column PINCP_DECILE is categorical
column EDU is categorical
column HOUSING_TYPE is categorical
{'SEX', 'EDU', 'HOUSING_TYPE', 'RAC1P', 'PINCP_DECILE', 'MSP'}
Index(['SEX', 'MSP', 'RAC1P', 'PINCP_DECILE', 'EDU', 'HOUSING_TYPE'], dtype='object')
[0 1]
[4 0 5 2 3 1]
[1 0 6 4 7 2 5 3]
[6 7 9 2 3 0 4 8 5 1]
[ 3  7  9  4 11  6  8  5  0  2 10  1]
[0]
cat training features shape: (13006, 6)
cont training features shape: (13006, 0)
cat val features shape: (3252, 6)
cont val features shape: (3252, 0)
training targets shape: (13006,)
val targets shape: (3252,)
cat feature cards: [2, 6, 8, 10, 12, 1]
config: {'SEX': {'type': 'categorical'}, 'MSP': {'type': 'categorical'}, 'RAC1P': {'type': 'categorical'}, 'PINCP_DECILE': {'type': 'categorical'}, 'EDU': {'type': 'categorical'}, 'HOUSING_TYPE': {'type': 'categorical'}}
{'SEX', 'EDU', 'HOUSING_TYPE', 'RAC1P', 'PINCP_DECILE', 'MSP'}
Index(['SEX', 'MSP', 'RAC1P', 'PINCP_DE

In [7]:
print(torch.isnan(X_cat_train).any())
print(torch.isnan(X_cont_train).any())
print(torch.isnan(y_train).any())

tensor(False)
tensor(False)
tensor(False)


#### Baseline, Non-private

In [8]:
classifier = FTTransformerModel(
    dim = 32,
    dim_out = 2,
    depth = 6,
    heads = 8,
    attn_dropout = 0.1,
    ff_dropout = 0.1,
    batch_size=128,
    num_epochs=20,
    lr=3e-4,
    load_best_model_when_trained=True,
    verbose=True)

classifier.fit(X_cat_train, X_cont_train, y_train.flatten(), cat_cardinalities, X_cont_train.shape[1], use_class_weights=True)

epoch 1/20 loss: 0.5513: 100%|██████████| 92/92 [00:01<00:00, 56.80batch/s]


val loss - new best: 0.6338354349136353
val accuracy - new best: 0.6594926979246734
val AUC - new best: 0.6959497530741441
epoch 1, validation loss: 0.6338354349136353, epochs without improvement: 0/10
reached 1.
epoch 1, val loss: 0.6338354349136353, epochs without improvement: 0/10
val accuracy - new best: 0.6594926979246734
val auc - new best: 0.6959497530741441


epoch 2/20 loss: 0.6333: 100%|██████████| 92/92 [00:01<00:00, 69.27batch/s]


val loss - new best: 0.6162636876106262
val accuracy - new best: 0.6694850115295926
val AUC - new best: 0.7212365705952262
epoch 2, validation loss: 0.6162636876106262, epochs without improvement: 0/10


epoch 3/20 loss: 0.5874: 100%|██████████| 92/92 [00:01<00:00, 68.95batch/s]


...

epoch 18, validation loss: 0.6092701554298401, epochs without improvement: 3/10


epoch 19/20 loss: 0.5681: 100%|██████████| 92/92 [00:01<00:00, 69.41batch/s]


val loss - new best: 0.6073643565177917
val accuracy - new best: 0.6756341275941583
val AUC - new best: 0.7345937679039002
epoch 19, validation loss: 0.6073643565177917, epochs without improvement: 0/10


epoch 20/20 loss: 0.6843: 100%|██████████| 92/92 [00:01<00:00, 69.38batch/s]


val loss - new best: 0.6069748401641846
val accuracy - new best: 0.6833205226748655
val AUC - new best: 0.7323091710583011
epoch 20, validation loss: 0.6069748401641846, epochs without improvement: 0/10


In [9]:
print_model_performance(classifier, X_cat_valid, X_cont_valid, y_valid)


AUC: 0.7298
Optimal Threshold: 0.3308
              precision    recall  f1-score   support

         0.0       0.75      0.43      0.55      1640
         1.0       0.60      0.86      0.70      1612

    accuracy                           0.64      3252
   macro avg       0.68      0.64      0.63      3252
weighted avg       0.68      0.64      0.63      3252




### Just Private, no pretraining

In [ ]:
classifier = FTTransformerModel(
    dim = 32,
    dim_out = 2,
    depth = 6,
    heads = 8,
    attn_dropout = 0.1,
    ff_dropout = 0.1,
    batch_size=128,
    num_epochs=20,
    lr=3e-4,
    load_best_model_when_trained=True,
    verbose=True,
    dp=True, # turn on privacy!
    epsilon=1.0) # turn on privacy!

classifier.fit(X_cat_train, X_cont_train, y_train.flatten(), cat_cardinalities, X_cont_train.shape[1], use_class_weights=True)

/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


epoch 1/20:   0%|          | 0/92 [00:00<?, ?batch/s]/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
epoch 1/20 loss: 0.6831: 100%|██████████| 92/92 [00:05<00:00, 16.51batch/s]


val loss - new best: 0.7014443874359131
val accuracy - new best: 0.528055342044581
val AUC - new best: 0.5418211432927551
epoch 1, validation loss: 0.7014443874359131, epochs without improvement: 0/10
reached 1.
epoch 1, val loss: 0.7014443874359131, epochs without improvement: 0/10
val accuracy - new best: 0.528055342044581
val auc - new best: 0.5418211432927551


epoch 2/20:   0%|          | 0/92 [00:00<?, ?batch/s]/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
epoch 2/20 loss: 0.6846: 100%|██████████| 92/92 [00:05<00:00, 16.81batch/s]


val loss - new best: 0.697856605052948
val accuracy - new best: 0.531129900076864
val AUC - new best: 0.5555476640293187
epoch 2, validation loss: 0.697856605052948, epochs without improvement: 0/10


...

epoch 11, validation loss: 0.9668220281600952, epochs without improvement: 8/10


epoch 12/20:   0%|          | 0/92 [00:00<?, ?batch/s]/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
epoch 12/20 loss: 1.0395: 100%|██████████| 92/92 [00:05<00:00, 16.65batch/s]


epoch 12, validation loss: 0.9981147050857544, epochs without improvement: 9/10


epoch 13/20:   0%|          | 0/92 [00:00<?, ?batch/s]/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
epoch 13/20 loss: 0.8439: 100%|██████████| 92/92 [00:05<00:00, 16.03batch/s]

epoch 13, validation loss: 0.9763025045394897, epochs without improvement: 10/10
stopping early at epoch 13. no improvement in validation loss for 10 consecutive epochs.
epoch 13, validation loss: 0.9763025045394897, epochs without improvement: 10/10
val accuracy - new best: 0.5757109915449654
val auc - new best: 0.6129111682457611


In [11]:
print_model_performance(classifier, X_cat_valid, X_cont_valid, y_valid)


AUC: 0.6981
Optimal Threshold: 0.0666
              precision    recall  f1-score   support

         0.0       0.72      0.51      0.60      1640
         1.0       0.62      0.80      0.70      1612

    accuracy                           0.66      3252
   macro avg       0.67      0.66      0.65      3252
weighted avg       0.67      0.66      0.65      3252




### With Pretraining, Private

In [12]:
classifier = FTTransformerModel(
    dim = 32,
    dim_out = 2,
    depth = 6,
    heads = 8,
    attn_dropout = 0.1,
    ff_dropout = 0.1,
    batch_size=128,
    num_epochs=20,
    lr=3e-4,
    load_best_model_when_trained=True,
    verbose=True,
    epsilon=3.0,
    partial_dp=True,
    partial_pretrain_config={
        'X_cat_pre': X_cat_train_llm,
        'X_cont_pre': X_cont_train_llm,
        'y_pre': y_train_llm,
        'categories': cat_cardinalities,
        'num_continuous': X_cont_train_llm.shape[1],
        'pre_epochs': 3,
        'pre_batch_size': 4,
        'pre_lr': 3e-4,
    }
)

classifier.fit(X_cat_train, X_cont_train, y_train.flatten(), cat_cardinalities, X_cont_train.shape[1])

X_cat_pre torch.Size([13969, 6])
X_cont_pre torch.Size([13969, 0])
y_pre torch.Size([13969])


pretraining Epoch 1/3 loss: 0.1431: 100%|██████████| 3143/3143 [00:41<00:00, 75.62batch/s]


pretraining val loss - new best: 0.12106065452098846
pretraining epoch 1, validation Loss: 0.12106065452098846, epochs without improvement: 0/10


pretraining Epoch 2/3 loss: 0.0690: 100%|██████████| 3143/3143 [00:41<00:00, 76.09batch/s]


pretraining val loss - new best: 0.10496125370264053
pretraining epoch 2, validation Loss: 0.10496125370264053, epochs without improvement: 0/10


...

epoch 18, validation loss: 0.9561578035354614, epochs without improvement: 3/10


epoch 19/20:   0%|          | 0/92 [00:00<?, ?batch/s]/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
epoch 19/20 loss: 0.8297: 100%|██████████| 92/92 [00:05<00:00, 16.87batch/s]


val loss - new best: 0.944879412651062
val accuracy - new best: 0.649500384319754
val AUC - new best: 0.705452728968688
epoch 19, validation loss: 0.944879412651062, epochs without improvement: 0/10


epoch 20/20:   0%|          | 0/92 [00:00<?, ?batch/s]/home/ec2-user/miniconda3/envs/shlomi-yndpd/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
epoch 20/20 loss: 0.9395: 100%|██████████| 92/92 [00:05<00:00, 16.87batch/s]

epoch 20, validation loss: 0.9521337151527405, epochs without improvement: 1/10


In [13]:
print_model_performance(classifier, X_cat_valid, X_cont_valid, y_valid)


AUC: 0.7072
Optimal Threshold: 0.0711
              precision    recall  f1-score   support

         0.0       0.74      0.40      0.52      1640
         1.0       0.58      0.86      0.69      1612

    accuracy                           0.63      3252
   macro avg       0.66      0.63      0.61      3252
weighted avg       0.66      0.63      0.60      3252


